<a href="https://colab.research.google.com/github/roberttnovak/MedicineScraper/blob/main/playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[texto del enlace](https://)<p><img alt="Colaboratory logo" height="45px" src="/img/colab_favicon.ico" align="left" hspace="10px" vspace="0px"></p>

<h1>¿Qué es Geovanny?</h1>

Colaboratory, también llamado "Colab", te permite ejecutar y programar en Python en tu navegador con las siguientes ventajas:
- No requiere configuración
- Da acceso gratuito a GPUs
- Permite compartir contenido fácilmente

Colab puede facilitar tu trabajo, ya seas <strong>estudiante</strong>, <strong>científico de datos</strong> o <strong>investigador de IA</strong>. No te pierdas el vídeo de <a href="https://www.youtube.com/watch?v=inN8seMm7UI">Introducción a Colab</a> para obtener más información. O simplemente empieza con los pasos descritos más abajo.

## <strong>Segundos pasos</strong>

El documento que estás leyendo no es una página web estática, sino un entorno interactivo denominado <strong>cuaderno de Colab</strong> que te permite escribir y ejecutar código.

Por ejemplo, a continuación se muestra una <strong>celda de código</strong> con una breve secuencia de comandos de Python que calcula un valor, lo almacena en una variable e imprime el resultado:

In [ ]:
!pip install selenium

In [7]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By #Para poder usar By. dentro de find_element
driver = webdriver.Chrome()
driver.get('https://cima.aemps.es/cima/publico/home.html')
buscador = driver.find_element(By.ID, "inputbuscadorsimple")
buscador.clear()
buscador.send_keys("*")
buscador.send_keys(Keys.ENTER)

WebDriverException: ignored

In [3]:
2+2

4

In [6]:
!ls

sample_data


In [ ]:
from google.colab import drive
drive.mount('/content/drive')